In [1]:
from rpy2.robjects.packages import importr
biobase = importr('Biobase')

In [2]:
eset = biobase.ExpressionSet()

In [3]:
type(eset)

rpy2.robjects.methods.RS4

In [4]:
tuple(eset.rclass)

('ExpressionSet',)

In [5]:
# print keys
print(tuple(eset.slots.keys()))

# fetch `phenoData`
phdat = eset.slots['phenoData']

# phdat is an S4 object itself
pheno_dataf = phdat.slots['data']

('.__classVersion__', 'experimentData', 'assayData', 'phenoData', 'featureData', 'annotation', 'protocolData', 'class')


In [6]:
from rpy2.robjects.methods import RS4   
class ExpressionSet(RS4):
    pass

eset_myclass = ExpressionSet(eset)

In [7]:
def ri2ro_s4(obj):
    if 'ExpressionSet' in obj.rclass:
        res = ExpressionSet(obj)
    else:
        res = robj
    return res

# try it
ri2ro_s4(eset)

<ExpressionSet - Python:0x7f193d757ec8 / R:0x3c38148>

In [8]:
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import Converter, localconverter

my_converter = Converter('ExpressionSet-aware converter',
                         template=default_converter)

from rpy2.rinterface import SexpS4
my_converter.ri2ro.register(SexpS4, ri2ro_s4)

<function __main__.ri2ro_s4>

In [9]:
with localconverter(my_converter) as cv:
    eset = biobase.ExpressionSet()
    print(type(eset))

<class '__main__.ExpressionSet'>


In [11]:
class ExpressionSet(RS4):
    def _exprs_get(self):
        return self.slots['assayData']
    def _exprs_set(self, value):
        self.slots['assayData'] = value
    exprs = property(_exprs_get,
                     _exprs_set,
                     None,
                     "R attribute `exprs`")
eset_myclass = ExpressionSet(eset)

eset_myclass.exprs

<Environment - Python:0x7f193d6dcc88 / R:0x3a83530>

In [17]:
from multipledispatch import dispatch
from functools import partial

my_namespace = dict()
dispatch = partial(dispatch, namespace=my_namespace)

@dispatch(ExpressionSet)
def rowmedians(eset,
               na_rm=False):
    res = biobase.rowMedians(eset,
                             na_rm=na_rm)
    return res

res = rowmedians(eset, na_rm=True)

NotImplementedError: Could not find signature for rowmedians: <ExpressionSet>

In [29]:

from multipledispatch import dispatch
from functools import partial

my_namespace = dict()
dispatch = partial(dispatch, namespace=my_namespace)

@dispatch(ExpressionSet)
def rowmedians(eset,
               na_rm=False):
    res = biobase.rowMedians(eset,
                             na_rm=na_rm)
    return res

res = rowmedians(eset)


NotImplementedError: Could not find signature for rowmedians: <ExpressionSet>

> /home/laurent/Desktop/software/python/py34_env/lib/python3.4/site-packages/multipledispatch/dispatcher.py(161)__call__()
    160                         'Could not find signature for %s: <%s>' %
--> 161                         (self.name, str_signature(types)))
    162             self._cache[types] = func

ipdb> l 150,30
    150 
    151 
    152     def __call__(self, *args, **kwargs):
    153         types = tuple([type(arg) for arg in args])
    154         try:
    155             func = self._cache[types]
    156         except KeyError:
    157             func = self.dispatch(*types)
    158             if not func:
    159                 raise NotImplementedError(
    160                         'Could not find signature for %s: <%s>' %
--> 161                         (self.name, str_signature(types)))
    162             self._cache[types] = func
    163         try:
    164             return func(*args, **kwargs)
    165 
    166         except MDNotImplementedError:
   

In [21]:
f=my_namespace['rowmedians']


In [23]:
f._cache


{}

In [25]:
@dispatch(int)
def rowmedians(eset,
               na_rm=False):
    res = biobase.rowMedians(eset,
                             na_rm=na_rm)
    return res

res = rowmedians(1)

/home/laurent/Desktop/software/python/py34_env/lib/python3.4/site-packages/rpy2-2.8.0.dev0-py3.4-linux-x86_64.egg/rpy2/robjects/functions.py:106: UserWarning: Error in (function (classes, fdef, mtable)  : 
  unable to find an inherited method for function ‘rowMedians’ for signature ‘"integer"’

  res = super(Function, self).__call__(*new_args, **new_kwargs)


RRuntimeError: Error in (function (classes, fdef, mtable)  : 
  unable to find an inherited method for function ‘rowMedians’ for signature ‘"integer"’


In [28]:
??f.dispatch

In [32]:
(type(eset), ) in f.funcs


False

In [33]:
f.funcs

{(__main__.ExpressionSet,): <function __main__.rowmedians>,
 (int,): <function __main__.rowmedians>}

In [34]:
type(eset)

__main__.ExpressionSet